# SUDOKU SOLVER

## PREPARING THE DEVELOPMENT ENVIRONMENT

Including path to previous directory in built-in variable `sys.path`, allowing access to the `sudoku` package:

In [1]:
import sys

sys.path.append('../')

Importing the relevant libraries:

In [2]:
import numpy as np
from collections import deque
from sudoku.utils import N, STEP, square_loc, objective_grid

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


## DEFINING CSP AND AC-3

Class that defines the specific CSP for sudoku and allows its resolution through a solver:

In [3]:
class SudokuCSP:
    def __init__(self, grid):
        self.domains     = self.generate_domains(grid)
        self.neighbors   = self.generate_neighbors()
        self.constraints = self.generate_constraints()
        self.cuts        = self.generate_cuts()

    def ac_3(self):
        queue = deque([(xi, xj)
                       for xi, xj in self.constraints
                       if self.assigned(xj)])

        while queue:
            xi, xj = queue.popleft()

            if self.revise(xi, xj):
                if self.assigned(xi):
                    queue.extend([(xk, xi)
                                  for xk in self.neighbors[xi] - {xj}])
                elif self.unfeasible(xi):
                    return False

        return True

    def revise(self, xi, xj):
        vj = self.domains[xj][0]

        for vi in self.domains[xi]:
            if vi == vj:
                self.domains[xi].remove(vi)
                return True

        return False

    def generate_domains(self, grid):
        return {(y, x) : self.get_domain(grid, y, x)
                for y in range(N)
                for x in range(N)}

    def generate_neighbors(self):
        return {pos : self.get_neighbors(*pos)
                for pos in self.vars}

    def generate_constraints(self):
        return [(v, n)
                for v in self.vars
                for n in self.neighbors[v]]

    def generate_cuts(self):
        return {v : []
                for v in self.vars}

    @staticmethod
    def get_domain(grid, y, x):
        return [grid[y, x]] if grid[y, x] else list(range(1, N+1))

    @staticmethod
    def get_row(y, x):
        return [(y, i)
                for i in range(N)
                if i != x]

    @staticmethod
    def get_col(y, x):
        return [(j, x)
                for j in range(N)
                if j != y]

    @staticmethod
    def get_sqr(y, x):
        return [(j, i)
                for j in range(*square_loc(y))
                for i in range(*square_loc(x))
                if j != y and i != x]

    def get_neighbors(self, y, x):
        return set(self.get_row(y, x) +
                   self.get_col(y, x) +
                   self.get_sqr(y, x))

    def unfeasible(self, xi):
        return len(self.domains[xi]) == 0

    def assigned(self, xi):
        return len(self.domains[xi]) == 1

    @property
    def vars(self):
        return self.domains.keys()

    @property
    def won(self):
        grid = np.zeros((N, N), dtype='int8')

        for pos, domain in self.domains.items():
            if len(domain) > 1:
                return False

            grid[pos] = domain[0]
        
        return objective_grid(grid)

Testing class instantiation and resolution through AC-3 (simpler instances are resolved only by propagating inferences):

In [4]:
%%timeit

grid = np.array([[1, 5, 7, 4, 0, 8, 9, 3, 0],
                 [8, 0, 3, 0, 0, 5, 0, 0, 4],
                 [6, 0, 4, 0, 1, 9, 8, 0, 7],
                 [0, 6, 0, 7, 0, 0, 0, 0, 0],
                 [0, 0, 0, 6, 0, 4, 2, 0, 3],
                 [0, 3, 9, 5, 0, 1, 0, 0, 0],
                 [5, 0, 1, 8, 0, 0, 3, 0, 9],
                 [3, 8, 0, 0, 5, 6, 0, 4, 0],
                 [0, 0, 6, 0, 3, 7, 0, 2, 0]], dtype='int8')

csp = SudokuCSP(grid)
csp.ac_3()

5.39 ms ± 474 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
grid = np.array([[1, 5, 7, 4, 0, 8, 9, 3, 0],
                 [8, 0, 3, 0, 0, 5, 0, 0, 4],
                 [6, 0, 4, 0, 1, 9, 8, 0, 7],
                 [0, 6, 0, 7, 0, 0, 0, 0, 0],
                 [0, 0, 0, 6, 0, 4, 2, 0, 3],
                 [0, 3, 9, 5, 0, 1, 0, 0, 0],
                 [5, 0, 1, 8, 0, 0, 3, 0, 9],
                 [3, 8, 0, 0, 5, 6, 0, 4, 0],
                 [0, 0, 6, 0, 3, 7, 0, 2, 0]], dtype='int8')

csp = SudokuCSP(grid)

print(f'Problem has a feasible solution = {csp.ac_3()}')
print(f'AC-3 was sufficient to resolve the instance of the problem = {csp.won}')

Problem has a feasible solution = True
AC-3 was sufficient to resolve the instance of the problem = True


## BACKTRACKING

Some instances of the sudoku game cannot be solved with AC-3 alone, and it is necessary to use some search strategy:

In [6]:
grid = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                 [8, 0, 0, 0, 0, 0, 0, 2, 0],
                 [0, 7, 0, 0, 1, 0, 5, 0, 0],
                 [4, 0, 0, 0, 0, 5, 3, 0, 0],
                 [0, 1, 0, 0, 7, 0, 0, 0, 6],
                 [0, 0, 3, 2, 0, 0, 0, 8, 0],
                 [0, 6, 0, 5, 0, 0, 0, 0, 9],
                 [0, 0, 4, 0, 0, 0, 0, 3, 0],
                 [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

csp = SudokuCSP(grid)

In [7]:
len_domain = 0
for domain in csp.domains.values():
    len_domain += len(domain)

print(f'Sum of the domain\'s option set before arc consistency is applied = {len_domain}')

Sum of the domain's option set before arc consistency is applied = 545


In [8]:
print(f'Problem has a feasible solution = {csp.ac_3()}')
print(f'AC-3 was sufficient to resolve the instance of the problem = {csp.won}')

Problem has a feasible solution = True
AC-3 was sufficient to resolve the instance of the problem = False


In [9]:
len_domain = 0
for domain in csp.domains.values():
    len_domain += len(domain)

print(f'Sum of domain option set after arc consistency is applied = {len_domain}')

Sum of domain option set after arc consistency is applied = 240


Implementing the heuristics and auxiliary functions used in backtracking search for csp:

In [10]:
def select_unassigned_variable(assignment, csp):
    available_vars = [var
                      for var in csp.vars
                      if var not in assignment.keys()]
    
    return min(available_vars, key=lambda var: len(csp.domains[var]))

def order_domain_values(assignment, csp, var):    
    lcv = lambda value: len([neighbor
                             for neighbor in csp.neighbors[var]
                             if neighbor not in assignment and value in csp.domains[neighbor]])

    return sorted(csp.domains[var], key=lcv)

def assign(assignment, csp, var, value):
    assignment[var] = value
    
    # Making inferences
    for neighbor in csp.neighbors[var]:
        if value in csp.domains[neighbor]:
            csp.domains[neighbor].remove(value)
            csp.cuts[var].append((neighbor, value))

def unassign(assignment, csp, var):
    # Unmaking inferences
    for neighbor, value in csp.cuts[var]:
        csp.domains[neighbor].append(value)

    csp.cuts[var].clear()

    del assignment[var]

def assignment_in_grid(assignment):
    grid = np.zeros((N, N), dtype='int8')

    for pos, value in assignment.items():
        grid[pos] = value

    return grid

Implementing backtracking search:

In [11]:
def backtracking_search(csp):
    assignment = {var : domain[0]
                  for var, domain in csp.domains.items()
                  if csp.assigned(var)}

    return backtrack(assignment, csp, len(csp.vars))

def backtrack(assignment, csp, vars):
    if len(assignment) == vars:
        return assignment

    var = select_unassigned_variable(assignment, csp)

    for value in order_domain_values(assignment, csp, var):
        assign(assignment, csp, var, value)

        result = backtrack(assignment, csp, vars)
        if result:
            return result

        unassign(assignment, csp, var)

    return None

Testing class instantiation and resolution via backtracking search:

In [12]:
assignment = backtracking_search(csp)
grid = assignment_in_grid(assignment)

print(f'Backtracking was sufficient to resolve the instance of the problem = {objective_grid(grid)}')

print()

print('Grid =')
print(grid)

Backtracking was sufficient to resolve the instance of the problem = True

Grid =
[[1 4 5 3 2 7 6 9 8]
 [8 3 9 6 5 4 1 2 7]
 [6 7 2 9 1 8 5 4 3]
 [4 9 6 1 8 5 3 7 2]
 [2 1 8 4 7 3 9 5 6]
 [7 5 3 2 9 6 4 8 1]
 [3 6 7 5 4 2 8 1 9]
 [9 8 4 7 6 1 2 3 5]
 [5 2 1 8 3 9 7 6 4]]


In [13]:
%%timeit

grid_inkala = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                        [8, 0, 0, 0, 0, 0, 0, 2, 0],
                        [0, 7, 0, 0, 1, 0, 5, 0, 0],
                        [4, 0, 0, 0, 0, 5, 3, 0, 0],
                        [0, 1, 0, 0, 7, 0, 0, 0, 6],
                        [0, 0, 3, 2, 0, 0, 0, 8, 0],
                        [0, 6, 0, 5, 0, 0, 0, 0, 9],
                        [0, 0, 4, 0, 0, 0, 0, 3, 0],
                        [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

csp = SudokuCSP(grid_inkala)
csp.ac_3()
backtracking_search(csp)

57.8 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
grid_inkala = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                        [8, 0, 0, 0, 0, 0, 0, 2, 0],
                        [0, 7, 0, 0, 1, 0, 5, 0, 0],
                        [4, 0, 0, 0, 0, 5, 3, 0, 0],
                        [0, 1, 0, 0, 7, 0, 0, 0, 6],
                        [0, 0, 3, 2, 0, 0, 0, 8, 0],
                        [0, 6, 0, 5, 0, 0, 0, 0, 9],
                        [0, 0, 4, 0, 0, 0, 0, 3, 0],
                        [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

csp = SudokuCSP(grid_inkala)
csp.ac_3()
assignment = backtracking_search(csp)
grid = assignment_in_grid(assignment)
    
print(f'Backtracking was sufficient to resolve the instance of the problem = {objective_grid(grid)}')

print()

print('Grid =')
print(grid)

Backtracking was sufficient to resolve the instance of the problem = True

Grid =
[[1 4 5 3 2 7 6 9 8]
 [8 3 9 6 5 4 1 2 7]
 [6 7 2 9 1 8 5 4 3]
 [4 9 6 1 8 5 3 7 2]
 [2 1 8 4 7 3 9 5 6]
 [7 5 3 2 9 6 4 8 1]
 [3 6 7 5 4 2 8 1 9]
 [9 8 4 7 6 1 2 3 5]
 [5 2 1 8 3 9 7 6 4]]
